In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io
import tensorflow as tf
from tensorflow.keras import Input, layers
from tensorflow.keras import models
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.lite.experimental.microfrontend.python.ops import audio_microfrontend_op as frontend_op
print(tf.__version__)
import numpy as np
import shutil, os
import matplotlib.pyplot as plt
import pathlib
import random
import sys
import io
import os
import glob
from datetime import datetime as dt


In [ ]:
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles

In [ ]:
IMG_SIZE=96 #image resolution is 96x96 dictated by pico
BATCH_SIZE = 64 #Batch for speed and regularization

### Deletes corrupted/unopenable files from source directory

In [ ]:
import os

num_skipped = 0
for folder_name in ("bird", "unknown"):
    folder_path = os.path.join("data/train", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)

### Get file count from source directory
> Bird directory used nested folders so getListOfFiles function used for that class

In [ ]:
bird_data_dir = '/Users/Jeromey/IOT/project2/data/birdNet'
bird_filenames = getListOfFiles(bird_data_dir)
len(bird_filenames)

unknown_data_dir = pathlib.Path(os.path.join('/Users/Jeromey/IOT/project2/data/imageNet/*'))
unknown_filenames = tf.io.gfile.glob(str(unknown_data_dir) + '/*')

print("Bird dataset size:", len(bird_filenames))
print("Unknown dataset size:", len(unknown_filenames))

In [ ]:
num_unknown=len(unknown_filenames)
random_unknown = list(range(0, len(unknown_filenames))) #create array [1,2,3...]
random.shuffle(random_unknown) #randomize the array
print("This will copy and move", num_unknown, "random files out of the", len(random_unknown),"available from the unknown dataset")

### For copying NotBird data
> Resizes to 96 x 96 and grayscale during copy

In [ ]:
#For copying unknown
num_files_to_copy=500000
error_count=0
for i in range(num_files_to_copy):
    try:
        shutil.copy2(unknown_filenames[random_unknown[i]],f'/Users/Jeromey/IOT/project2/data/train/unknown')
        os.rename(f'/Users/Jeromey/IOT/project2/data/train/unknown/'+os.path.basename(unknown_filenames[random_unknown[i]]),
                  f'/Users/Jeromey/IOT/project2/data/train/unknown/'+str(f"{i}{i}.jpeg")
                 )
    except:
        error_count=error_count+1
        print("Something went wrong when copying",error_count,"times")

### For copying Bird data
> Resizes to 96 x 96 and grayscale during copy

In [ ]:
#For copying bird
error_count=0
num_files_to_copy=len(bird_filenames)
for i in range(num_files_to_copy):
    try:
        shutil.copy2(bird_filenames[i],f'/Users/Jeromey/IOT/project2/data/train/bird')
        os.rename(f'/Users/Jeromey/IOT/project2/data/train/bird/'+os.path.basename(bird_filenames[i]),
                  f'/Users/Jeromey/IOT/project2/data/train/bird/'+str(f"{i}.jpeg")
                 )
    except:
        error_count=error_count+1
        print("Something went wrong when copying",error_count,"times")

### Createing a data_generator to augment data

> Creating extreme zoom and flip to imitate camera transition

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io

# Construct an instance of the ImageDataGenerator class
# Pass the augmentation parameters through the constructor. 

notbird_datagen = ImageDataGenerator(
        rotation_range=90,     #Random rotation between 0 and 90
        width_shift_range=0.2,   #% shift
        height_shift_range=0.3,
        shear_range=0.4,
        zoom_range=0.8,
        horizontal_flip=True,
        fill_mode='reflect',
        )    

In [ ]:
#runs through loop and augments entire dataset # of loops
i = 0
for batch in notbird_datagen.flow_from_directory(directory='data/train/n', 
                                         batch_size=84900,  
                                         target_size=(96, 96),
                                         color_mode="grayscale",                                        
                                         save_to_dir='/Users/Jeromey/IOT/project2/data/augmented/notperson', 
                                         save_prefix='aug', 
                                         save_format='jpeg'):
    i += 1
    if i > 2: #makes 2 augmentations 
        break 

> Minimal augmentation for Bird dataset

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from skimage import io

# Construct an instance of the ImageDataGenerator class
# Pass the augmentation parameters through the constructor. 

bird_datagen = ImageDataGenerator(
        rotation_range=25,     #Random rotation between 0 and 45
        width_shift_range=0.1,   #% shift
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.1,
        horizontal_flip=True,
        fill_mode='reflect',
        )    #Also try nearest, constant, reflect, wrap

In [ ]:
#runs through loop and augments entire dataset # of loops
i = 0
for batch in bird_datagen.flow_from_directory(directory='data/train/p', 
                                         batch_size=84900,  
                                         target_size=(96, 96),
                                         color_mode="grayscale",
                                         save_to_dir='/Users/Jeromey/IOT/project2/data/augmented/bird', 
                                         save_prefix='aug', 
                                         save_format='jpeg'):
    i += 1
    if i > 2: #makes 2 augmentations 
        break 

### Used to augment all data in directory
>resize and change to grayscale

In [ ]:
dataset = []

import numpy as np
from skimage import io
import os
from PIL import Image

image_directory = 'data/augmented/'
SIZE = 96
dataset = []

my_images = os.listdir(image_directory)
for i, image_name in enumerate(my_images):
    if (image_name.split('.')[1] == 'jpg'):
        image = io.imread(image_directory + image_name)
        image = Image.fromarray(image, 'grayscale')
        image = image.resize((SIZE,SIZE))
        dataset.append(np.array(image))

x = np.array(dataset)

i = 0
for batch in datagen.flow(x, batch_size=16,  
                          save_to_dir='data/testout/', 
                          save_prefix='aug', 
                          save_format='jpeg'):
    i += 1
    if i > 20:
        break  